**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

### Sequence Diagram

```mermaid

%%{
  init: {
    'theme': 'base',
    'themeVariables': {
      'primaryColor': '#ffffff',
      'primaryTextColor': '#000000',
      'primaryBorderColor': '#000000',
      'lineColor': '#000000',
      'secondaryColor': '#f4f4f4',
      'secondaryTextColor': '#000000',
      'tertiaryColor': '#d3d3d3',
      'tertiaryTextColor': '#000000',
      'background': '#ffffff',
      'actorBkg': '#B4B4B4',
      'actorTextColor': '#000000',
      'actorBorder': '#000000',
      'actorLineColor': '#000000',
      'signalColor': '#000000',
      'signalTextColor': '#000000',
      'activationBorderColor': '#000000',
      'activationBkgColor': '#d3d3d3',
      'sequenceNumberColor': '#000000',
      'noteBkgColor': '#F0F0F0',
      'noteTextColor': '#000000',
      'noteBorderColor': '#000000'
    }
  }
}%%

sequenceDiagram
    participant Platform as "Platform"
    participant Blockchain as "Iroha 1 Blockchain"

    Note over Platform, Blockchain: Deploy smart contract for account creation
    Platform->>Blockchain: Deploy Smart Contract
    Blockchain-->>Platform: Smart Contract Deployed Successfully
    
    Note over Platform, Blockchain: Account creation
    Platform->>Blockchain: Create Account
    Blockchain-->>Platform: Account Created Successfully

    Note over Platform, Blockchain: Queries the blockchain to confirm account creation
    Platform->>Blockchain: Query Account Details
    Blockchain-->>Platform: Query Response
    
    
    Note over Platform, Blockchain: Deploy smart contract for details setting
    Platform->>Blockchain: Deploy Smart Contract
    Blockchain-->>Platform: Smart Contract Deployed Successfully

    Note over Platform, Blockchain: Set user details
    Platform->>Blockchain: Set User Details in Blockchain
    Blockchain-->>Platform: User Details Set Successfully

    Note over Platform, Blockchain: Queries the blockchain to confirm details setting
    Platform->>Blockchain: Query Account Details
    Blockchain-->>Platform: Query Response
```

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

In [1]:
import os
import binascii
# import csv
from iroha import IrohaCrypto
from iroha import Iroha, IrohaGrpc
import sys
from Crypto.Hash import keccak
import integration_helpers
from name_generator import left, right
import json
from utilities import *
from dump_to_json import *
from iroha_helper import *

if sys.version_info[0] < 3:
    raise Exception("Python 3 or a more recent version is required.")

# Load configuration from config.json file
config_path = "config.json"  # Update this path as needed
with open(config_path, "r") as f:
    config = json.load(f)

IROHA_HOST_ADDR = config["IROHA_HOST_ADDR"]
IROHA_PORT = config["IROHA_PORT"]
ADMIN_ACCOUNT_ID = config["ADMIN_ACCOUNT_ID"]
ADMIN_PRIVATE_KEY = config["ADMIN_PRIVATE_KEY"]
iroha = Iroha(ADMIN_ACCOUNT_ID)
net = IrohaGrpc("{}:{}".format(IROHA_HOST_ADDR, IROHA_PORT))


# - Deploy contract
hash = create_account_contract()



2025-01-11 18:43:17.929 | INFO     | integration_helpers:tracer:34 - 	Entering "create_account_contract"
2025-01-11 18:43:17.936 | INFO     | iroha_helper:create_account_contract:44 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-11 18:43:17.936 | INFO     | iroha_helper:create_account_contract:44 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-11 18:43:19.596 | INFO     | iroha_helper:create_account_contract:44 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-11 18:43:19.623 | INFO     | iroha_helper:create_account_contract:44 - ('COMMITTED', 5, 0)
2025-01-11 18:43:19.625 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_account_contract"


In [2]:
DOMAIN = "test"

2 - Creates a user account into the blockchain.

In [3]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

@integration_helpers.trace
def get_account(address):
    params = integration_helpers.get_first_four_bytes_of_keccak(b"getAccount(string)")
    no_of_param = 1
    for x in range(no_of_param):
        params = params + integration_helpers.left_padded_address_of_param(
            x, no_of_param
        )
    params = params + integration_helpers.argument_encoding(f"{user_account_short_id}@{DOMAIN}")  # account id
    tx = iroha.transaction(
        [
            iroha.command(
                "CallEngine", caller=ADMIN_ACCOUNT_ID, callee=address, input=params
            )
        ]
    )
    IrohaCrypto.sign_transaction(tx, ADMIN_PRIVATE_KEY)
    response = net.send_tx(tx)
    for status in net.tx_status_stream(tx):
        logger.info(status)
    hex_hash = binascii.hexlify(IrohaCrypto.hash(tx))
    return hex_hash


# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
address = integration_helpers.get_engine_receipts_address(hash)
create_account(address, user_account_short_id, DOMAIN, user_public_key, user_account)

# - Get account info
hash = get_account(address)
address = integration_helpers.get_engine_receipts_result(hash)


2025-01-11 18:43:19.699 | INFO     | __main__:<module>:37 - User Role: author
2025-01-11 18:43:19.701 | INFO     | __main__:<module>:48 - Creating account with name: sweet_matsumoto
2025-01-11 18:43:19.702 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-11 18:43:19.721 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_address"
2025-01-11 18:43:19.723 | INFO     | integration_helpers:tracer:34 - 	Entering "create_account"
2025-01-11 18:43:19.745 | INFO     | iroha_helper:create_account:84 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-11 18:43:19.747 | INFO     | iroha_helper:create_account:84 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)


2025-01-11 18:43:22.653 | INFO     | iroha_helper:create_account:84 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-11 18:43:22.684 | INFO     | iroha_helper:create_account:84 - ('COMMITTED', 5, 0)
2025-01-11 18:43:22.689 | INFO     | dump_to_json:dump_to_json_ld:76 - Appended new entry to file 'datasets/accounts.json', current total entries: 21
2025-01-11 18:43:22.691 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_account"
2025-01-11 18:43:22.692 | INFO     | integration_helpers:tracer:34 - 	Entering "get_account"
2025-01-11 18:43:22.711 | INFO     | __main__:get_account:23 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-11 18:43:22.712 | INFO     | __main__:get_account:23 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-11 18:43:25.713 | INFO     | __main__:get_account:23 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-11 18:43:25.740 | INFO     | __main__:get_account:23 - ('COMMITTED', 5, 0)
2025-01-11 18:43:25.744 | INFO     | integration_helpers:tracer:36 - 	Leaving

3 - Queries the blockchain to confirm the succesful creation of the account.

In [4]:
#Query - GetAccountDetail

query = iroha.query('GetAccountDetail',account_id=f"{user_account_short_id}@{DOMAIN}")
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
data = response.account_detail_response
logger.info(f'Account id = {user_account_short_id}@{DOMAIN}, details = {data.detail}')

2025-01-11 18:43:25.811 | INFO     | __main__:<module>:7 - Account id = sweet_matsumoto@test, details = {}


4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting.

In [5]:
# - Deploys the contract
hash = create_detail_contract()
address = integration_helpers.get_engine_receipts_address(hash)
integration_helpers.get_engine_receipts_result(hash)

2025-01-11 18:43:25.864 | INFO     | integration_helpers:tracer:34 - 	Entering "create_detail_contract"
2025-01-11 18:43:25.886 | INFO     | iroha_helper:create_detail_contract:58 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-11 18:43:25.888 | INFO     | iroha_helper:create_detail_contract:58 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)


2025-01-11 18:43:28.763 | INFO     | iroha_helper:create_detail_contract:58 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-11 18:43:28.794 | INFO     | iroha_helper:create_detail_contract:58 - ('COMMITTED', 5, 0)
2025-01-11 18:43:28.796 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_detail_contract"
2025-01-11 18:43:28.798 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-11 18:43:28.817 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_address"
2025-01-11 18:43:28.819 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_result"
2025-01-11 18:43:28.838 | INFO     | integration_helpers:get_engine_receipts_result:92 - 
2025-01-11 18:43:28.839 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_result"


5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key

In [6]:
process_account(address, f"{user_account_short_id}@{DOMAIN}")


2025-01-11 18:43:28.858 | INFO     | iroha_helper:process_account:292 - {'@type': 'foaf:Person', 'foaf:name': 'Sweet Matsumoto', 'foaf:mbox': 'sweet_matsumoto@email.com', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Technological University (Loikaw)'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '7800-4857-2654-X'}, 'foaf:holdsAccount': {'schema:identifier': 'sweet_matsumoto@test', 'schema:roleName': 'author', 'schema:publicKey': '5e010bc63eef08374f9bf38028c5c0eaa65dd5d18dbf51ad77b5fd68ec0c6fe2'}}
2025-01-11 18:43:28.889 | INFO     | iroha_helper:process_account:296 - user_json_ld_cid :, QmeV2diLkJCctiwM6i81yo6qkWBZfoRj1xsRRXdALejupF
2025-01-11 18:43:28.890 | INFO     | iroha_helper:tracer:218 - 	Entering "set_account_detail"
2025-01-11 18:43:28.907 | INFO     | iroha_helper:set_account_detail:270 - None
2025-01-11 18:43:28.909 | INFO     | iroha_helper:set_account_detail:272 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-11 18:

6 - Queries the blockchain again to confirm the succesfull details setting.

In [ ]:
#Query - GetAccountDetail

query = iroha.query('GetAccountDetail',account_id=f"{user_account_short_id}@{DOMAIN}")
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
data = response.account_detail_response
details = data.detail
logger.info(details)
logger.info(f'User Account id = {user_account_short_id}@{DOMAIN}, {details}')

2025-01-11 18:43:31.880 | INFO     | __main__:<module>:8 - { "admin@test" : { "user_json_ld_cid" : "QmeV2diLkJCctiwM6i81yo6qkWBZfoRj1xsRRXdALejupF" } }
2025-01-11 18:43:31.881 | INFO     | __main__:<module>:9 - User Account id = sweet_matsumoto@test, { "admin@test" : { "user_json_ld_cid" : "QmeV2diLkJCctiwM6i81yo6qkWBZfoRj1xsRRXdALejupF" } }


7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [8]:
# Convert the JSON string to a Python dictionary
details_dict = json.loads(details)

# Now you can access the specific key like this
user_metadata_cid = details_dict["admin@test"]["user_json_ld_cid"]
logger.info(user_metadata_cid)

user_metadata = download_json_from_ipfs(user_metadata_cid)

logger.info(user_metadata)

2025-01-11 18:43:31.900 | INFO     | __main__:<module>:6 - QmeV2diLkJCctiwM6i81yo6qkWBZfoRj1xsRRXdALejupF
2025-01-11 18:43:31.908 | INFO     | __main__:<module>:10 - {'@type': 'foaf:Person', 'foaf:holdsAccount': {'schema:identifier': 'sweet_matsumoto@test', 'schema:publicKey': '5e010bc63eef08374f9bf38028c5c0eaa65dd5d18dbf51ad77b5fd68ec0c6fe2', 'schema:roleName': 'author'}, 'foaf:mbox': 'sweet_matsumoto@email.com', 'foaf:name': 'Sweet Matsumoto', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Technological University (Loikaw)'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '7800-4857-2654-X'}}
